# BeautifulSoup - 정적 웹페이지 스크래핑

In [ ]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup 

In [ ]:
requests.get("https://www.naver.com/")

In [ ]:
# request.get 함수로 서버에 응답 요청 (존재하는 페이지)
url = "https://www.python.org/"
resp = requests.get(url)
print(resp)

In [ ]:
# request.get 함수로 서버에 응답 요청 (존재하지 않는 페이지)
url2 = "https://www.python.org/1"
resp2 = requests.get(url2)
print(resp2)

## 뉴스 서비스에 접속

In [ ]:
# 뉴스 사이트
url = "https://news.daum.net/"

# User-agent 정보
agent = 'Mozila/2.0'

# requests.get 함수로 서버에 요청
resp = requests.get(url, headers={'User-agent': agent})
print(resp)

In [ ]:
# HTML 소스코드
resp.text

In [ ]:
# HTML 소스코드 해석
soup = BeautifulSoup(resp.text, 'html.parser')
print(type(soup))

In [ ]:
# BeautifulSoup 객체 (HTML 소스코드 해석 결과를 저장하고 있음)
soup

In [ ]:
# head 태그 출력
print(soup.head)

In [ ]:
# body 태그 출력 
print(soup.body)

In [ ]:
# title 태그 검색
print('title 태그 요소: ', soup.title)
print('title 태그 이름: ', soup.title.name)
print('title 태그 문자열: ', soup.title.text)

## find 메서드, find_all 메서드
- [Ctrl] + [Shift] + "i"
- 개발자 도구 열기

### ul 태그

In [ ]:
# find - 가장 먼저 나타나는 태그를 찾습니다
soup.find(name='ul')

In [ ]:
# find_all - 모든 태그를 찾습니다
ul_data = soup.find_all(name='ul')
len(ul_data)

In [ ]:
# 첫 번째 ul 태그를 출력
ul_data[0]

### 태그 속성

In [ ]:
# class 속성이 "list_newsissue"인 ul 태그를 모두 찾는다
newsissue = soup.find_all(name='ul', attrs={'class':'list_newsissue'})
len(newsissue)

In [ ]:
print(type(newsissue))
print(type(newsissue[0]))

In [ ]:
# 첫번째 태그 내용
newsissue[0]

## select 메서드

### ul 태그

In [ ]:
soup.select("ul")

In [ ]:
ul_list = soup.select('ul')
len(ul_list)

### 클래스 속성자

In [ ]:
# class 속성값이 list_newsissue인 경우
class_list = soup.select('.list_newsissue')
len(class_list)

In [ ]:
# class_list[0]안에 들어 있는 li 태그들
li_list = soup.select('ul.list_newsissue > li')
len(li_list)

### ID 속성자

In [ ]:
# id="kakaoServiceLogo"

id_list = soup.select('#kakaoServiceLogo')
len(id_list)

In [ ]:
id_list[0]

## [실습] 
li_list에 들어 있는 20개의 뉴스 중에서 하나를 골라서, 뉴스 제목/뉴스 카테고리/언론사 이름/뉴스 링크를 정리합니다.

In [ ]:
# 첫 번째 뉴스를 담고 있는 태그
li_list[0]

In [ ]:
# 뉴스 제목 - select
li_list[0].select("a.link_txt")[0].text.strip()

In [ ]:
# 뉴스 제목 - find 
li_list[0].find('a', attrs={'class':'link_txt'}).text.strip()

In [ ]:
# 뉴스 링크 - select
li_list[0].select("a.link_txt")[0]["href"]

In [ ]:
# 뉴스 링크 - find
li_list[0].find('a', attrs={'class':'link_txt'})['href']

In [ ]:
# 언론사 이름 
[t['alt'] for t in li_list[0].find_all('img', attrs={'class':'thumb_g'}) if t['alt'] != ''][0]

In [ ]:
# 뉴스 카테고리 - select
li_list[0].select("span.txt_category")[0].text.strip()

In [ ]:
# 뉴스 카테고리 - find
li_list[0].find('span', attrs={'class':'txt_category'}).text.strip()

## [실습] li_list에 들어 있는 20개의 뉴스를 반복문을 이용하여 하나의 데이터프레임으로 정리합니다.

In [ ]:
# 데이터를 담을 딕셔너리 객체
data = {'title':[], 'agency':[], 'category':[], 'link':[]}

# 객 태그를 반복문으로 순회하면서 아이템별로 추출해서 딕서너리 객체의 리스트 원소에 추가 
for item in li_list:

    data['title'].append(item.find('a', attrs={'class':'link_txt'}).text.strip())
    data['link'].append(item.find('a', attrs={'class':'link_txt'})['href'])
    try:
        data['agency'].append([t['alt'] for t in item.find_all('img', attrs={'class':'thumb_g'}) if t['alt'] != ''][0])
    except:
        data['agency'].append(item.find('span', attrs={'class':'thumb_g'}).text.strip())
        
    data['category'].append(item.find('span', attrs={'class':'txt_category'}).text.strip())

# 추출된 딕셔너리 객체를 출력해서 확인 
data

In [ ]:
for k in data.keys():
    print(len(data[k]))

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df

In [ ]:
# csv 파일로 저장
df.to_csv('news.csv', index=False)